# Meta Classes

In [1]:
# Basic MetaClasses

class MyMeta(type):
    def __new__(cls, name: str, bases: tuple, dct: dict) -> type:
        dct['meta_attribute'] = "Added by metaclass"
        return super().__new__(cls, name, bases, dct)

class MyClass(metaclass=MyMeta):
    pass

instance = MyClass()
print(instance.meta_attribute)


Added by metaclass


In [3]:
# Restrict Class names

class CapitalizeNameMeta(type):
    def __new__(cls, name: str, bases: tuple, dct: dict) -> type:
        if not name[0].isupper():
            raise ValueError("Class name must start with an uppercase letter")
        return super().__new__(cls, name, bases, dct)

class MyClass(metaclass=CapitalizeNameMeta):
    pass

class myclass(metaclass=CapitalizeNameMeta):  # ValueError
    pass


ValueError: Class name must start with an uppercase letter

In [4]:
# Singleton example using Meta Class

class SingletonMeta(type):
    _instances: dict[str, object] = {}

    def __call__(cls, *args, **kwargs) -> object:
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class SingletonClass(metaclass=SingletonMeta):
    def __init__(self) -> None:
        self.value = 42

# Testing Singleton
obj1 = SingletonClass()
obj2 = SingletonClass()
print(obj1 is obj2)  # Output: True
print(obj1.value)
print(obj2.value)

True
42
42


In [6]:
# Logging Class Creation:

import logging

logging.basicConfig(level=logging.INFO)

class LogMeta(type):
    def __new__(cls, name: str, bases: tuple, dct: dict) -> type:
        logging.info(f"Creating class: {name}")
        return super().__new__(cls, name, bases, dct)

class LoggedClass(metaclass=LogMeta):
    pass

type(LoggedClass)
# Output: INFO:root:Creating class: LoggedClass


__main__.LogMeta

In [7]:
# Adding Methods dynamically with Metaclass

class MethodAdderMeta(type):
    def __new__(cls, name: str, bases: tuple, dct: dict) -> type:
        def dynamic_method(self) -> str:
            return f"Hello from {name}"

        dct['dynamic_method'] = dynamic_method
        return super().__new__(cls, name, bases, dct)

class MyClass(metaclass=MethodAdderMeta):
    pass

instance = MyClass()
print(instance.dynamic_method())


Hello from MyClass


In [8]:
# Customize Inheritance using meta Classes

class InheritanceMeta(type):
    def __new__(cls, name: str, bases: tuple, dct: dict) -> type:
        if 'required_method' not in dct:
            raise TypeError(f"{name} must implement 'required_method'")
        return super().__new__(cls, name, bases, dct)

class BaseClass:
    def required_method(self) -> str:
        return "Implemented"

class DerivedClass(BaseClass, metaclass=InheritanceMeta):
    pass

#This would raise an error
class InvalidClass(metaclass=InheritanceMeta):
    pass  # TypeError: InvalidClass must implement 'required_method'


TypeError: DerivedClass must implement 'required_method'

# Design Principles

### Single Responsibility Principle

In [11]:
# Before applying SRP
class Employee:
    def __init__(self, name: str, salary: float) -> None:
        self.name = name
        self.salary = salary

    def calculate_payroll(self) -> float:
        return self.salary * 0.75

    def save_to_db(self) -> None:
        print(f"Saving {self.name} to database.")

# After applying SRP
class Employee:
    def __init__(self, name: str, salary: float) -> None:
        self.name = name
        self.salary = salary

class PayrollCalculator:
    def calculate_payroll(self, employee: Employee) -> float:
        return employee.salary * 0.75

class DatabaseSaver:
    def save(self, employee: Employee) -> None:
        print(f"Saving {employee.name} to database.")

# Example usage
employee = Employee("Hemanth", 50000)
payroll = PayrollCalculator().calculate_payroll(employee)
DatabaseSaver().save(employee)


Saving Hemanth to database.


In [15]:
import requests
from typing import Any


class FileDownloader:
    """
    Downloader: Responsible for fetching file data.
    """
    def download(self, url: str) -> bytes:
        response = requests.get(url)
        response.raise_for_status()
        return response.content


class FileParser:
    def parse(self, data: bytes) -> Any:
      """
      Parser: Responsible for converting raw data to structured form.
      """
      return data.decode('utf-8').splitlines()

url = "https://raw.githubusercontent.com/dwyl/english-words/master/words.txt"
downloader = FileDownloader()
parser = FileParser()
file_data = downloader.download(url)
lines = parser.parse(file_data)
print("First 5 lines of the parsed file:")
print("\n".join(lines[:5]))


First 5 lines of the parsed file:
2
1080
&c
10-point
10th


### Open-Closed Principle

In [20]:
from math import pi
# not open-closed (Violation)

class Shape:
    def __init__(self, shape_type: str, **kwargs) -> None:
        self.shape_type = shape_type
        if self.shape_type == "circle":
            self.radius = kwargs["radius"]
        elif self.shape_type == "rectangle":
            self.length = kwargs["length"]
            self.width = kwargs["width"]

    def area(self) -> float:
        if self.shape_type == "circle":
            return pi * self.radius ** 2
        elif self.shape_type == "rectangle":
            return self.length * self.width

def main() -> None:
    shapes = [
        Shape("circle", radius=5),
        Shape("rectangle", length=4, width=6)
    ]
    for shape in shapes:
        print(f"Area of {shape.shape_type}: {shape.area()}")


if __name__ == "__main__":
    main()

Area of circle: 78.53981633974483
Area of rectangle: 24


In [28]:
# Demonstration of Open-Closed Principle

from abc import ABC, abstractmethod
from math import pi

class Shape(ABC):
    @abstractmethod
    def area(self) -> float:
        pass

class Circle(Shape):

    def __init__(self, radius: float) -> None:
        self.radius = radius

    def area(self) -> float:
        return pi * self.radius

class Rectangle(Shape):

    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

    def area(self) -> float:
        return self.width * self.height

class Square(Rectangle):

    def __init__(self, side_length: float) -> None:
        super().__init__(side_length, side_length)

    def area(self) -> float:
        return super().area()

class Triangle(Shape):

    def __init__(self, base: float, height: float) -> None:
        self.base = base
        self.height = height

    def area(self) -> float:
        return 0.5 * self.base * self.height

def main() -> None:

    shapes = [
        Circle(5),
        Rectangle(4, 6),
        Square(3),
        Triangle(3, 4)
    ]

    for shape in shapes:
        print(f"Area of {type(shape).__name__}: {shape.area()}")

if __name__ == "__main__":
    main()




Area of Circle: 15.707963267948966
Area of Rectangle: 24
Area of Square: 9
Area of Triangle: 6.0


In [27]:
# Demonstration of Open-Closed Principle

from abc import ABC, abstractmethod

class PaymentMethod(ABC):

    @abstractmethod
    def pay(self, amount: float) -> None:
        pass

class CreditCardPayment(PaymentMethod):

    def pay(self, amount: float) -> None:
        print(f"Processing credit card payment of {amount} rupees")

class UpiPayment(PaymentMethod):

    def pay(self, amount: float) -> None:
        if amount > 100000:
            print("Upi Payment can't be over 100,000 rupees")
        else:
            print(f"Processing Upi payment of {amount} rupees")

class BitcoinPayment(PaymentMethod):

    def pay(self, amount: float) -> None:
        print(f"Processing Bitcoin payment of {amount} rupees")

payments = [CreditCardPayment(), UpiPayment(), BitcoinPayment()]
for payment in payments:
    payment.pay(300000)


Processing credit card payment of 300000 rupees
Upi Payment can't be over 100,000 rupees
Processing Bitcoin payment of 300000 rupees


### Liskov Substitution Principle

In [29]:
# Liskov Substitution Violation

# Incorrect design example:
class Bird:
    def fly(self) -> None:
        print("Flying")

class Penguin(Bird):
    def fly(self) -> None:
        raise NotImplementedError("Penguins cannot fly")

def make_bird_fly(bird: Bird) -> None:
    bird.fly()

make_bird_fly(Penguin())


NotImplementedError: Penguins cannot fly

In [30]:
# Correct the violation

class Bird:

    def move(self) -> None:
        print("Moving")

class FlyingBird(Bird):

    def fly(self) -> None:
        print("Flying")

class Eagle(FlyingBird):

    def fly(self) -> None:
        print("Flying high")

class Penguin(Bird):

    def move(self) -> None:
        print("Walking")

def make_bird_fly(bird: FlyingBird) -> None:

    bird.fly()

eagle = Eagle()
make_bird_fly(eagle)
penguin = Penguin()
penguin.move()


Flying high
Walking


In [32]:
# Demonstration of Liskov Substituion Principle


class Rectangle:
    """
    # Base Rectangle class:
    """
    def __init__(self, width: float, height: float) -> None:
        self._width = width
        self._height = height

    @property
    def width(self) -> float:
        return self._width

    @width.setter
    def width(self, value: float) -> None:
        self._width = value

    @property
    def height(self) -> float:
        return self._height

    @height.setter
    def height(self, value: float) -> None:
        self._height = value

    def area(self) -> float:
        return self._width * self._height


class Square(Rectangle):
    """
     A Square can be implemented carefully to preserve LSP.
    """

    def __init__(self, side: float) -> None:
        super().__init__(side, side)

    @Rectangle.width.setter
    def width(self, value: float) -> None:
        self._width = value
        self._height = value

    @Rectangle.height.setter
    def height(self, value: float) -> None:
        self._width = value
        self._height = value

def print_area(rect: Rectangle) -> None:
    print(f"Area: {rect.area()}")

rect = Rectangle(4, 5)
square = Square(4)
print_area(rect)
print_area(square)


Area: 20
Area: 16


### Interface Segmentation Principle

In [33]:
# Without ISP

from abc import ABC, abstractmethod

class Worker(ABC):
    @abstractmethod
    def work(self) -> None:
        pass

    @abstractmethod
    def eat(self) -> None:
        pass

class HumanWorker(Worker):
    def work(self) -> None:
        print("Human working...")

    def eat(self) -> None:
        print("Human eating...")

class RobotWorker(Worker):
    def work(self) -> None:
        print("Robot working...")

    def eat(self) -> None:
        raise NotImplementedError("Robots don't eat")


In [34]:
# ISP

from abc import ABC, abstractmethod

class Workable(ABC):
    @abstractmethod
    def work(self) -> None:
        pass

class Eatable(ABC):
    @abstractmethod
    def eat(self) -> None:
        pass

class HumanWorker(Workable, Eatable):
    def work(self) -> None:
        print("Human working...")

    def eat(self) -> None:
        print("Human eating...")

class RobotWorker(Workable):
    def work(self) -> None:
        print("Robot working...")

human: Workable = HumanWorker()
robot: Workable = RobotWorker()
human.work()
if isinstance(human, Eatable):
    human.eat()
robot.work()


Human working...
Human eating...
Robot working...


In [36]:
# Multifunction Printer

from abc import ABC, abstractmethod

class Printer(ABC):

    @abstractmethod
    def print_document(self, document: str) -> None:
        pass

class Scanner(ABC):

    @abstractmethod
    def scan_document(self) -> str:
        pass

class MultiFunctionPrinter(Printer, Scanner):

    def print_document(self, document: str) -> None:
        print(f"Printing document: {document}")

    def scan_document(self) -> str:
        scanned = "Scanned document content"
        print(scanned)
        return scanned

class SimplePrinter(Printer):

    def print_document(self, document: str) -> None:
        print(f"Printing document: {document}")

mfp: Printer = MultiFunctionPrinter()
mfp.print_document("Test Document")
sp: Printer = SimplePrinter()
sp.print_document("Another Document")


Printing document: Test Document
Printing document: Another Document


### Dependency Inversion Principle

In [38]:
# Before DIP

# Not recommended
class FileManager:

    def read(self) -> str:
        with open('/content/sample_data/file.txt', 'r') as file:
            return file.read()

class Report:

    def __init__(self) -> None:
        self.file_manager = FileManager()

    def generate_report(self) -> str:
        return self.file_manager.read()


In [39]:
# After DIP

from abc import ABC, abstractmethod

class Reader(ABC):
    @abstractmethod
    def read(self) -> str:
        pass

class FileReader(Reader):

    def read(self) -> str:
        with open('/content/sample_data/file.txt', 'r') as file:
            return file.read()

class Report:
    def __init__(self, reader: Reader) -> None:
        self.reader = reader

    def generate_report(self) -> str:
        return self.reader.read()

with open('/content/sample_data/file.txt', 'w') as f:
    f.write("This is a sample report content.")

reader: Reader = FileReader()
report = Report(reader)
print("Report Content:")
print(report.generate_report())



Report Content:
This is a sample report content.
